In [ ]:
# ~~~~~~~~ Importing necessary Libraries ~~~~~~~~
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import re
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from geopy.geocoders import Nominatim
from sklearn.neural_network import MLPRegressor

# ~~~~~~~~ Initializing Actual and Predicted Distances and Times as Lists ~~~~~~~~
DistanceFromLocation = []
TimeToLocation = []
predicted_DistanceFromLocation = []
predicted_TimesFromLocation = []


# ~~~~~~~~ Using Nominatim's route finder to get coordinates based off of addresses that the user inputs into the system ~~~~~~~~
geoLocation = Nominatim(user_agent='route_finder')

# ~~~~~~~~ Function to get coordinates from the address ~~~~~~~~
def patientAddress_ToCoordinates(address):
    patientsAddress = geoLocation.geocode(address)
    if patientsAddress is None:
        print(f'Could not get coordinates for: {address}. Re-enter the patient\'s address.')
        return None
    return patientsAddress.latitude, patientsAddress.longitude

# ~~~~~~~~ Function to prioritize patients based on severity and distance from hospital ~~~~~~~~   
def patientPrioritization(patients):
    Patient_SeverityOrder = {
        'Critical': 0,
        'Severe': 1,
        'Non-Severe': 2
    }
    return sorted(patients, key=lambda x: (Patient_SeverityOrder[x['Severity']], x['Distance']))

# ~~~~~~~~ Function that uses Euclidean Distance to calculate the distance between two coordinates ~~~~~~~~ 
def euclidean_calculation(coordinate_1, coordinate_2):
    return ((coordinate_1[0] - coordinate_2[0]) ** 2 + (coordinate_1[1] - coordinate_2[1]) ** 2) ** 0.5

# ~~~~~~~~ Getting the edges and nodes of Laredo, Texas from the dataset folder (CSV Files) and dropping any NaN values that are in it ~~~~~~~~
edgesDataFrame = pd.read_csv('datasets/edges.csv')
nodesDataFrame = pd.read_csv('datasets/nodes.csv')
edgesDataFrame = edgesDataFrame.dropna()
nodesDataFrame = nodesDataFrame.dropna()

# ~~~~~~~~ Creating graph from Laredo and adding edges ~~~~~~~~
Graph = ox.graph_from_place('Laredo, Texas, USA', network_type='all')
for _, row in edgesDataFrame.iterrows():
    # ~~~~~~~~ Adding various attributes to the Graph such as oneway, highway, and the weight of the edge and making the length the weight for the Graph ~~~~~~~~ 
    Graph.add_edge(row['u'], row['v'], row['oneway'], row['highway'], weight=row['length'])

# ~~~~~~~~ Creating a list for patients and asking the user to input the amount of patients that will be visited / delivered to ~~~~~~~~
patients = []
totalOfPatients = int(input('Enter the number of patients that need to be delivered: '))

for _ in range(totalOfPatients):
    nameOfPatient = input('Enter the patient\'s name: ')
    addressOfPatient = input('Enter the patient\'s address: ')
    severityOfPatient = input('Enter the severity of the patient: (Critical, Severe, Non-Severe): ')
    patientCoords = patientAddress_ToCoordinates(addressOfPatient)

    if patientCoords:
        patients.append({'Name': nameOfPatient, 
                         'Address': addressOfPatient,
                         'Severity': severityOfPatient,
                         'Coordinates': patientCoords})

# ~~~~~~~~ Initializing the hospitals location (HEB Pharmacy) and converting the address to become coordinates ~~~~~~~~
HospitalLocation = '1301 Guadalupe St, Laredo, Texas, USA'
HospitalCoords = patientAddress_ToCoordinates(HospitalLocation)

# ~~~~~~~~ Preparing features and targets for ML model ~~~~~~~~
featuresToBeUsed = []
desired_Distance = []
desired_Time = []

# ~~~~~~~~ Creating the features and targets for the Neural Network Regression model ~~~~~~~~
for patient in patients:
    distance = euclidean_calculation(HospitalCoords, patient['Coordinates'])
    severity = patientPrioritization.get('Severity', 'Non-Severe')
    featuresToBeUsed.append([severity, distance])
    desired_Distance.append(distance)
    desired_Time.append(distance / 50)

X = np.array(featuresToBeUsed)
y_Distance = np.array(desired_Distance)
y_Time = np.array(desired_Time)

#                                           ~~~~~~~~                                  Neural Network Regressor Machine Learning                ~~~~~~~~

model_distance = MLPRegressor(hidden_layer_sizes=(1000, 1000), max_iter=1000, random_state=42)
model_time = MLPRegressor(hidden_layer_sizes=(1000, 1000), max_iter=1000, random_state=42)

model_distance.fit(X, y_Distance)
model_time.fit(X, y_Time)

previousDestinationNode = ox.distance.nearest_nodes(Graph, HospitalCoords[1], HospitalCoords[0])

print("\n~~~~~~~~ Summary of Routes ~~~~~~~~")
for index, patient in enumerate(patients):
    # ~~~~~~~~ Getting the coordinates of the patient and making in the coordinates for the destination ~~~~~~~~
    destinationCoords = patient['Coordinates']
    if destinationCoords:
        try:

            # ~~~~~~~~ Pathing the path to the patient using Bidirectional Djikstras Algorithm ~~~~~~~~
            destinationNode = ox.distance.nearest_nodes(Graph, destinationCoords[1], destinationCoords[0])
            if previousDestinationNode is not None and destinationNode is not None:
                shortestPath = nx.bidirectional_dijkstra(Graph, previousDestinationNode, destinationNode, weight='weight')[1]
                
                total_Distance = 0
                total_Time = 0


                # ~~~~~~~~ Calculating the total distance and time of the route for the patient and making sure that any missing data is entered with a default value ~~~~~~~~
                for u, v in zip(shortestPath[:-1], shortestPath[1:]):
                    data_Edge = Graph.get_edge_data(u, v)
                    data_Length = data_Edge[0].get('length', 0)
                    
                    maxSpeed_String = data_Edge[0].get('maxspeed', 50)
                    try:
                        max_speed = float(maxSpeed_String)
                    except ValueError:
                        max_speed = 50

                    maxSpeed_mininum = (max_speed * 1000) / 60
                    data_Time = data_Length / maxSpeed_mininum

                    total_Distance += data_Length
                    total_Time += data_Time

                # ~~~~~~~~  Necessary conversions for distance and time ~~~~~~~~

                # ~~~~~~~~ The value of 1609.34 is the conversion factor for meters to miles
                total_distance_miles = total_Distance / 1609.34

                # ~~~~~~~~ Predicting the distance and the time from the route using the Neural Network Regressor Machine Learning ~~~~~~~~
                predicted_DistanceFromRoute = model_distance.predict([[patientPrioritization[patient['Severity']], total_distance_miles]])[0]
                predicted_TimeFromRoute = model_time.predict([[patientPrioritization[patient['Severity']], total_distance_miles]])[0]
                predicted_DistanceFromLocation.append(predicted_DistanceFromRoute)
                predicted_TimesFromLocation.append(predicted_TimeFromRoute)
                DistanceFromLocation.append(total_distance_miles)
                TimeToLocation.append(total_Time)


                # ~~~~~~~~ Output for Patients ~~~~~~~~
                print(f"Route to {patient['Name']} (Severity: {patient['Severity']})")
                print(f"Total distance: {total_distance_miles:.2f} miles")
                print(f"Estimated travel time: {total_Time:.2f} minutes")

                if predicted_DistanceFromLocation and len(DistanceFromLocation) == len(predicted_DistanceFromLocation):
                    meanABS_Distance = mean_absolute_error(DistanceFromLocation, predicted_DistanceFromLocation)
                    meanABS_Time = mean_absolute_error(TimeToLocation, predicted_TimesFromLocation)
                    
                    rootMSE_Distance = np.sqrt(mean_squared_error(DistanceFromLocation, predicted_DistanceFromLocation))
                    rootMSE_Time = np.sqrt(mean_squared_error(TimeToLocation, predicted_TimesFromLocation))
                    
                    r2_Distance = r2_score(DistanceFromLocation, predicted_DistanceFromLocation)
                    r2_Time = r2_score(TimeToLocation, predicted_TimesFromLocation)
                    
                    print(f"\n~~~~~~~~ Evaluation Metrics ~~~~~~~~")
                    print(f"Mean Absolute Error (Distance): {meanABS_Distance:.2f} miles")
                    print(f"Mean Absolute Error (Time): {meanABS_Time:.2f} minutes")
                    print(f"Root Mean Squared Error (Distance): {rootMSE_Distance:.2f} miles")
                    print(f"Root Mean Squared Error (Time): {rootMSE_Time:.2f} minutes")

                # ~~~~~~~~ Plotting the routes and showing the shortest path between the pharmacy and the patients ~~~~~~~~
                fig, ax = ox.plot_graph_route(
                    Graph, shortestPath, route_linewidth=6, node_size=0, bgcolor='white', show=False, close=False
                )

                marginForGraph = 0.01
                ax.set_xlim(destinationCoords[1] - marginForGraph, destinationCoords[1] + marginForGraph)
                ax.set_ylim(destinationCoords[0] - marginForGraph, destinationCoords[0] + marginForGraph)

                ax.plot(HospitalCoords[1], HospitalCoords[0], 'o', markersize=10, label='Hospital/Pharmacy', color='blue')
                for p in patients:
                    ax.plot(p['Coordinates'][1], p['Coordinates'][0], 'o', markersize=5, label=p['Name'], color='green')

                ax.legend()
                ax.set_title(f'Route to {patient["Name"]}')

                plt.show()

                previousDestinationNode = destinationNode
            else:
                print(f'Invalid nodes for {patient["Name"]}')
        except nx.NetworkXNoPath:
            print(f'No path found to {patient["Address"]}')
